### Problem Statement:
##### Classifying the msrp of car prices

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### Data Acquisition


In [56]:
url = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

In [57]:
df = pd.read_csv(url)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

##### Features
##### For the rest of the homework, you'll need to use only these columns:

##### (Make,  Model,Year,Engine HP, Engine Cylinders, Transmission Type, Vehicle Style, highway MPG, city mpg,MSRP)

##### Select only the features from above and transform their names using the next line:
##### Fill in the missing values of the selected features with 0.
##### Rename MSRP variable to price.

### Data Preparation

In [59]:
df = df[['Make','Model','Year','Engine HP', 
              'Engine Cylinders', 'Transmission Type', 
              'Vehicle Style', 'highway MPG', 'city mpg','MSRP']]

In [60]:
df

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg,MSRP
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


In [67]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'msrp'],
      dtype='object')

In [68]:
df.rename(columns = {'msrp': 'price'})
df.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'msrp'],
      dtype='object')

### Question 1
##### What is the most frequent observation (mode) for the column transmission_type?

##### AUTOMATIC
##### MANUAL
#### AUTOMATED_MANUAL
#### DIRECT_DRIVE

In [73]:
df.transmission_type[df.transmission_type.value_counts().max()]

'AUTOMATIC'

### Question 2
###### Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

###### What are the two features that have the biggest correlation in this dataset?

###### engine_hp and year
###### engine_hp and engine_cylinders
###### highway_mpg and engine_cylinders
###### highway_mpg and city_mpg



### Make price binary
###### Now we need to turn the price variable from numeric into a binary format.
###### Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.

### Split the data
###### Split your data in train/val/test sets with 60%/20%/20% distribution.
###### Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
###### Make sure that the target value (above_average) is not in your dataframe.